In [1]:
from aip import AipBodyAnalysis
import cv2
import time
from PIL import Image  
import os
import threading

In [30]:
def get_file_content(filePath):
    with open(filePath,'rb') as fp:
        return fp.read()
    


def style_transfer(model,
                    img,
                   pathOut='',
                   width=None,
                   jpg_quality=80,
                    ):
    ## 从本地加载预训练模型
    print('加载预训练模型......')
    #print(pathIn)
    ## 读入原始图片，调整图片至所需尺寸，然后获取图片的宽度和高度
    #img = cv2.imread(pathIn)
    (h, w) = img.shape[:2]
    if width is not None:
        img = cv2.resize(img, (width, round(width*h/w)), interpolation=cv2.INTER_CUBIC)
        (h, w) = img.shape[:2]

    ## 将图片构建成一个blob：设置图片尺寸，将各通道像素值减去平均值（比如ImageNet所有训练样本各通道统计平均值）
    ## 然后执行一次前馈网络计算，并输出计算所需的时间
    blob = cv2.dnn.blobFromImage(img, 1.0, (w, h), (103.939, 116.779, 123.680), swapRB=False, crop=False)
    model.setInput(blob)
    start = time.time()
    output = model.forward()
    end = time.time()
    print("风格迁移花费：{:.2f}秒".format(end - start))

    ## reshape输出结果, 将减去的平均值加回来，并交换各颜色通道
    output = output.reshape((3, output.shape[2], output.shape[3]))
    output[0] += 103.939
    output[1] += 116.779
    output[2] += 123.680
    output = output.transpose(1, 2, 0)
    ## 输出风格化后的图片
    cv2.imwrite(pathOut, output, [int(cv2.IMWRITE_JPEG_QUALITY), jpg_quality])

def get_model(rootdir):
    list = os.listdir(rootdir) #列出文件夹下所有的目录与文件
    models=[]
    for i in list:
        i = rootdir+'/'+i
        model = cv2.dnn.readNetFromTorch(i)
        models.append(model)
    
    return models   

def get_pic(rootdir):
    list = os.listdir(rootdir) #列出文件夹下所有的目录与文件
    pics=[]
    for i in list:
        i = rootdir+'/'+i
        pics.append(i)
    
    return pics   

def get_finger_num():
    print('a')
    img=get_file_content('a.jpg')
    #print(img)
    a=client.gesture(img)
    #print(a)
    if(len(a)==3):
        if(a['result']!=[]):
            if (a['result'][0]['classname']=='Fist'):
                print('ok1')
                return 0
            if (a['result'][0]['classname']=='One'):
                print('ok2')
                return 1
            if (a['result'][0]['classname']=='Two'):
                print('ok3')
                return 2
            if (a['result'][0]['classname']=='Three' or a['result'][0]['classname']=='Ok'):
                print('ok4')
                return 3
            if (a['result'][0]['classname']=='Four'):
                print('ok5')
                return 4
            if (a['result'][0]['classname']=='Five'):
                print('ok6')
                return 5



In [31]:
API_ID = '17631993'                                    #新的
API_KEY = 'Lui5fo5NNOXkVRszGtnVAm5t'
SECRET_KEY = 'MLAhhrrUegGEN0zlXTnQYq4KcW5b0tSC'
client = AipBodyAnalysis(API_ID,API_KEY,SECRET_KEY)  # 建立连接  
models=get_model('D:\jupyter\homework\model')

In [33]:
import time
pic_flag=0
show_flag=0
finger_flag=0
camera_finger=[None]
camera_pic=[None]
camera = cv2.VideoCapture(1)
camera1 = cv2.VideoCapture(0)
img=cv2.imread('D:\jupyter\homework\data/shuai2.png')
cv2.imshow('lalal',img)
cv2.waitKey(3000)
cv2.namedWindow("lalal",cv2.WINDOW_NORMAL)
while 1:
    if finger_flag==1:
        finger_flag=0
        #print('1')
        finger_num=get_finger_num()
        if finger_num==0:
            break
        if finger_num==5:
            pic_flag=1
        if finger_num==2:
            show_flag=1
            
    if pic_flag==1:
        pic_flag=0
        show_flag=1
        a=int(time.time())
        for i in range(len(models)):
            pathOut="D:\jupyter\homework\save/"+str(a)+str(i)+'.jpg'
            output=style_transfer(models[i],camera_frame1[0],pathOut)
                
    if show_flag==1:
        show_flag=0
        pic_paths=get_pic('D:\jupyter\homework\save')
        pics=get_pic('D:\jupyter\homework\pic')
        #print(pic_paths)
        #print(len(pic_paths))
        for i in range(7):
            a=len(pic_paths)-1-i
            frame1=camera_pic[0]
            #frame1=cv2.imread(pic_paths[a])
            frame2=cv2.imread(pic_paths[a])
            frame3=cv2.imread(pics[i])
            img[0:479,0:639] = frame1[0:479,0:639]
            img[0:479,640:1279]= frame2[0:479,0:639]
            img[480:719,0:1279]= frame3[0:239,0:1279]
            cv2.imshow('lalal',img)
            k=cv2.waitKey(2000)
                       
    if camera.isOpened():
        ret1, frame1= camera.read()
        ret2,frame2 = camera1.read()
        camera_pic[0]=frame2
        camera_finger[0]=frame1
        cv2.imwrite('a.jpg',frame1)
        #print(frame1)
        frame3=cv2.imread('D:\jupyter\homework\data/finger.png')
        img[0:479,0:639] = frame1[0:479,0:639]
        #img[0:479,640:1279]= frame1[0:479,0:639]
        img[0:479,640:1279]= frame2[0:479,0:639]
        img[480:719,0:1279]= frame3[0:239,0:1279]
        cv2.imshow('lalal',img)
        k=cv2.waitKey(10)
        if k == 27:
            finger_flag=1

camera.release()
cv2.destroyAllWindows() 

a
ok6
加载预训练模型......
风格迁移花费：1.21秒
加载预训练模型......
风格迁移花费：0.55秒
加载预训练模型......
风格迁移花费：0.53秒
加载预训练模型......
风格迁移花费：0.56秒
加载预训练模型......
风格迁移花费：0.72秒
加载预训练模型......
风格迁移花费：0.54秒
加载预训练模型......
风格迁移花费：0.55秒
a
ok3
a
ok1


In [21]:
camera.release()
cv2.destroyAllWindows() 